# RVC 1, Ch2.2
https://petercorke.github.io/spatialmath-python/func_3d.html

In [1]:
# Works best with jupyter-notebook

In [2]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np

from spatialmath import *
from spatialmath.base import *
from roboticstoolbox import *
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

## Lec 3.12 Quaternions
https://petercorke.github.io/spatialmath-python/func_quat.html

## Quaternions

A quaternion is often described as a type of complex number but it is more useful (and simpler) to think of it as an order pair comprising a scalar and a vector.  We can create a quaternion object as follows:

In [4]:
q1 = Quaternion([1,1,0,0])
q1
type(q1)

spatialmath.quaternion.Quaternion

where the scalar is before the angle brackets which enclose the vector part.  

Properties allow us to extract the scalar part

In [5]:
q1.s

1

and the vector part

In [6]:
q1.v

array([1, 0, 0])

and we can represent it as a numpy array

In [7]:
q1.vec

array([1, 1, 0, 0])

A quaternion has a conjugate

In [8]:
q1.conj()

 1.0000 < -1.0000, -0.0000, -0.0000 >


and a norm, which is the magnitude of the equivalent 4-vector 

In [9]:
q1.norm()

1.4142135623730951

We can create a second quaternion

In [10]:
q2 = Quaternion([1,0,1,0])
q2

 1.0000 <  0.0000,  1.0000,  0.0000 >


Operators allow us to add

In [11]:
q1 + q2

 2.0000 <  1.0000,  1.0000,  0.0000 >


subtract

In [12]:
q1 - q2

 0.0000 <  1.0000, -1.0000,  0.0000 >


and to multiply

In [14]:
q1 * q2

 1.0000 <  1.0000,  1.0000,  1.0000 >


which,  follows the special rules of Hamilton multiplication.

Multiplication can also be performed as the linear algebraic product of one quaternion converted to a 4x4 matrix  

In [15]:
q1.matrix

array([[       1,       -1,        0,        0],
       [       1,        1,        0,        0],
       [       0,        0,        1,       -1],
       [       0,        0,        1,        1]])

and the other as a 4-vector 

In [16]:
q1.matrix @ q2.vec

array([       1,        1,        1,        1])

The product of a quaternion and its conjugate is a **scalar** equal to the square of its norm

In [17]:
q1 * q1.conj()

 2.0000 <  0.0000,  0.0000,  0.0000 >


Conversely, a quaternion with a zero scalar part is called a _pure quaternion_

In [18]:
Quaternion.Pure([1, 2, 3])

 0.0000 <  1.0000,  2.0000,  3.0000 >


### Unit quaternions

A quaternion with a unit norm is called a _unit quaternion_ .  

It is a group and its elements represent **rotation in 3D space**.  It is in all regards like an $\mbox{SO}(3)$ matrix except for a _double mapping_ -- a quaternion and its element-wise negation represent the same rotation.

In [21]:
q1 = UnitQuaternion.Rx(60, 'deg')
print(q1)

import math
print('\n',math.cos( math.radians(30) ) )
print(math.sin( math.radians(30) ) )

 0.8660 <<  0.5000,  0.0000,  0.0000 >>

 0.8660254037844387
0.49999999999999994


The convention is that unit quaternions are denoted using double angle brackets.  The norm, as advertised is indeed one

In [22]:
q1.norm()

1.0

We create another unit quaternion

In [23]:
q2 = UnitQuaternion.Rx(-60, 'deg')
q2

 0.8660 << -0.5000,  0.0000,  0.0000 >>


Rotations can be composed by quaternion multiplication. As if q1 was acting on q2.

In [24]:
q3 = q1 * q2
q3

 1.0000 <<  0.0000,  0.0000,  0.0000 >>


We can convert a quaternion to a rotation matrix

In [25]:
q3.R

array([[       1,        0,        0],
       [       0,        1,        0],
       [       0,        0,        1]])

which yields exactly the same answer as if we'd done it using SO(3) rotation matrices

In [26]:
SO3.Rx(60, 'deg') * SO3.Rx(-60, 'deg')

SO3:   1           0           0            
       0           1           0            
       0           0           1            
    

The advantages of unit quaternions are that

1. they are compact, just 4 numbers instead of 9
2. multiplication involves fewer operations and is therefore faster
3. numerical errors build up when we multiply rotation matrices together many times, and they lose the structure (the columns are no longer unit length or orthogonal).  Correcting this, the process of _normalization_ is expensive.  

For unit quaternions errors will also compound, but normalization is simply a matter of dividing through by the norm

Unit quaternions have an inverse

In [27]:
q2.inv()

 0.8660 <<  0.5000, -0.0000, -0.0000 >>


In [28]:
q1 * q2.inv()

 0.5000 <<  0.8660,  0.0000,  0.0000 >>


or

In [29]:
q1 / q2

 0.5000 <<  0.8660,  0.0000,  0.0000 >>


We can convert any unit quaternion to an SO3 object if we wish

In [30]:
q1.SO3()

SO3:   1           0           0            
       0           0.5        -0.866025     
       0           0.866025    0.5          
    

and conversely, any `SO3` object to a unit quaternion

In [31]:
UnitQuaternion( SO3.Rx(60, 'deg'))

 0.8660 <<  0.5000,  0.0000,  0.0000 >>


A unit quaternion is not a minimal representation. 

Since we know the magnitude is 1, then with any 3 elements we can compute the fourth upto a sign ambiguity. 

In [32]:
q1.vec3


array([     0.5,        0,        0])

In [33]:
a = UnitQuaternion.qvmul( q1.vec3, q2.vec3)
a

array([       0,        0,        0])

from which we can recreate the unit quaternion

In [34]:
UnitQuaternion.Vec3(a)

 1.0000 <<  0.0000,  0.0000,  0.0000 >>
